In [ ]:
#https://nlpforhackers.io/wordnet-sentence-similarity/
#https://realpython.com/python-csv/
#import re
import csv
import json
#import ijson
#import bisect
import gensim
import nltk
#import _thread
#import threading
#from multiprocessing import Pool
from multiprocessing.pool import ThreadPool
#from threading import Thread
#import multiprocessing
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import itertools
import scipy.interpolate as interpolate
from operator import itemgetter
from scipy import stats
from itertools import groupby
from collections import Counter
#from gensim.models import LdaModel
#from stop_words import get_stop_words
from nltk import pos_tag
from IPython.display import clear_output
#from nltk.corpus import stopwords as nltk_stop_words, wordnet as wn
#from nltk.tokenize import RegexpTokenizer
#from nltk.stem.porter import PorterStemmer
#from scipy.interpolate import spline
#from sklearn.feature_extraction.text import TfidfVectorizer
folder_path = 'D:/MS CS/RS DATA/dataset/StarRank/'
print("success")

### Co-authors number of common papers (Just Count)
#### -> Read authors id
#### -> Check their number of co-authors and build coupus

In [ ]:
author_aid = []
with open(folder_path+'author-corpus.json','r', encoding="utf8") as f:
    for doc in f:
        author_aid.append(json.loads(doc)['aid'])
    f.close()
    
    coauthor_common_paper = pd.read_csv(folder_path+'AMiner-Coauthor.txt', sep='\t', names=['aid', 'co_aid', 'common_paper'])
    coauthor_common_paper['aid'] = (coauthor_common_paper['aid'].str.replace('#','')).astype(str).astype(int)
    coauthor_common_paper = coauthor_common_paper[coauthor_common_paper['aid'].isin(author_aid)]
    coauthor_common_paper.to_csv(folder_path+'coauthor_common_paper.csv', sep=',', encoding='utf-8', index=False)
    coauthor_common_paper.head()
    print("Success...")

In [ ]:
author_to_paper = pd.read_json(folder_path+'author-2-paper.json', orient='records', encoding='utf8', lines=True)
author_to_paper_pid_grouped = author_to_paper.groupby("pid")
unique_author = list(set(author_to_paper['aid']))
print("Success...")

In [ ]:
outfile_acw = open(folder_path+'acw.csv', 'w+', encoding='utf8', newline='')
acw_writer = csv.writer(outfile_acw)

def calWSC(first_auth):
    for second_auth in unique_author:
        if first_auth != second_auth:
            data = []
            ac_lk = 0.0
            pac_k = 0.0
            for pid, doc in author_to_paper_pid_grouped:
                aid = list(doc['aid'])
                second_auth_aid = False
                if second_auth in aid:
                    pac_k += (1/int(doc[doc.aid == second_auth]['pos']))
                    second_auth_aid = True
                if second_auth_aid and first_auth in aid:
                    ac_lk += (1/int(doc[doc.aid == first_auth]['pos']))
                    ac_lk += (1/int(doc[doc.aid == second_auth]['pos']))
                
            clear_output()
            print('first_auth',first_auth,'second_auth', second_auth, 'ACW', (ac_lk/pac_k))
            data.append(first_auth)
            data.append(second_auth)
            data.append((ac_lk/pac_k))
            acw_writer.writerow(data)

if __name__ == "__main__":
    
    acw_writer.writerow(['first_auth', 'second_auth', 'acw'])
    pool = ThreadPool(16)
    pool.map(calWSC, unique_author)
    pool.close()
    pool.join()
    #pd.DataFrame(author_score).to_json(folder_path+'acw.json', orient='records', lines=True)
    print('success')

In [ ]:
labels = ["a","a","a","b","b","b"]

stats.entropy(list(Counter(labels).values()), base=2)

In [ ]:
prs_corpus = pd.read_json(folder_path+'prs-corpus.json', orient='records', encoding='utf8', lines=True)

In [ ]:
unList(prs_corpus[prs_corpus.id==821833]['title'])

In [ ]:
with open(folder_path+'rs-corpus.json', 'r', encoding='utf8') as f:
    rs_corpus = []
    for doc in f:
        data = json.loads(doc)
        del data['authors']
        del data['vas']
        del data['year']
        data['title'] = list(chain(*prs_corpus[prs_corpus.id==data['id']]['title']))
        rs_corpus.append(data)
    f.close()
    rs_corpus = pd.DataFrame(rs_corpus)
    rs_corpus.to_json(folder_path+'star-rs-corpus.json', orient='records', lines=True)
print("success")

In [ ]:
prs_corpus = []
with open(folder_path+'preprocessed-corpus.json', 'r', encoding='utf8') as f:
    for doc in f:
        data = json.loads(doc)
        for year in data:
            for rec in data[year]:
                prs_corpus.append(rec)
    f.close()
    prs_corpus = pd.DataFrame(prs_corpus)
    #prs_corpus.to_json(folder_path+'star-rs-corpus.json', orient='records', lines=True)
    print ("preprocessed-corpus successfully loaded...")

In [ ]:
print(list(chain(*data['title'])))